In [18]:
%matplotlib inline
import pandas as pd
from bs4 import BeautifulSoup

train = pd.read_csv('labeledTrainData.tsv',header=0,delimiter='\t',quoting=3)
ex1 = BeautifulSoup(train["review"][0])
#print train.info
#print train["review"][0]
#print ex1.get_text()

For the sake of simplicity we're going to trim down the reviews to just letters, ignoring symbols and numbers for now. To do this we'll use the regex library re and then we'll tokenise the reviews.

In [29]:
import re

# Use regular expressions to do a find-and-replace:
# Find anything that is NOT a lowercase letter (a-z)
# or an upper case letter (A-Z), and replace it with a space

letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      ex1.get_text() )  # The text to search
print letters_only[:50] + '\n'

# Now do tokenisation : convert to all lowercase and run .split

lower_case = letters_only.lower()        # Convert to lower case
words = lower_case.split()               # Split into words

print words[:20]

 With all this stuff going down at the moment with

[u'with', u'all', u'this', u'stuff', u'going', u'down', u'at', u'the', u'moment', u'with', u'mj', u'i', u've', u'started', u'listening', u'to', u'his', u'music', u'watching', u'the']


Now we'll get rid of stopwords with the NLTK library

In [ ]:
import nltk
#nltk.download()  # Download text data sets, including stop words
# Remove stop words from "words"
words = [w for w in words if not w in stopwords.words("english")]
print words